In [38]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import time
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from pymorphy2 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iraid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Скачивание описаний вакансий

## С использованием API ΗΗ

#### Формирование списка id вакансий для последующего скачивания описаний и прочих параметров каждой из них

In [4]:
page=0
per_page=100


url=f'https://api.hh.ru/vacancies?per_page={per_page}&page={page}'

res=requests.get(url)

vacancies=res.json()

vacancies_id=set({})

while 'items' in vacancies.keys():
    for i in range(len(vacancies['items'])):
        vacancies_id.add(vacancies['items'][i]['id'])
    
    page+=1
    
    url=f'https://api.hh.ru/vacancies?per_page={per_page}&page={page}'
    
    res=requests.get(url)

    vacancies=res.json()
    
    #print(url)

vacancies_id=list(vacancies_id)

In [5]:
print('Количество вакансий: {0}'.format(len(vacancies_id)))

Количество вакансий: 2000


#### Сохранение списка id вакансий

In [6]:
#f=open(r'D:\Дима\Учеба\ML\DA.SberUniversity\Final project\hh_vacancies_id.txt','w')
#for vac in vacancies_id:
    #f.write(str(vac)+'\n')
#f.close()

#### Сбрка и предобработка описаний вакансий

In [7]:
# функция, удаляющая все символы, кроме букв, цифр, пробелов (а также группы пробелов), подчеркиваний, и приводящая слова к нормальной форме
def cleaner_1(s):
    
    morph=MorphAnalyzer()
    
    s=re.sub('\W',' ',s)
    clean=[]
    for token in s.split(' '):
        if token!='':
            clean.append(morph.normal_forms(token)[0])
    return ' '.join(clean)    

cnt_vac=len(vacancies_id)


#собираем из вакансий необходимую информацию

agg_description=[]


for i in range(len(vacancies_id[:cnt_vac])):
    
    
    url_one=f'https://api.hh.ru/vacancies/{vacancies_id[i]}'

    res_one=requests.get(url_one)

    vacancy=res_one.json()

    def ValExist(d,k):
        if k in d.keys() and d[k] is not None:
            return True
        else:
            return False

    if ValExist(vacancy,'name'):
        name=vacancy['name']
    else:
        name=''

    if ValExist(vacancy,'area'):
        area=vacancy['area']['name']
    else:
        area=''

    if ValExist(vacancy,'salary'):
        rep=lambda x: 'рубль' if x=='RUR' else 'валюта'
        currency=rep(vacancy['salary']['currency'])
    else:
        currency=''

    if ValExist(vacancy,'address'):
        if ValExist(vacancy['address'],'city'):
            city = vacancy['address']['city']
        else:
            city=''
        if ValExist(vacancy['address'],'street'):
            street = vacancy['address']['street']
        else:
            street=''
        if ValExist(vacancy['address'],'building'):
            building = vacancy['address']['building']
        else:
            building=''
        
        address= city + ' ' + street + ' ' + building
    else:
        address=''

    if ValExist(vacancy,'experience'):
        experience = vacancy['experience']['name']
    else:
        experience=''

    if ValExist(vacancy,'experience'):
        schedule=vacancy['schedule']['name']
    else:
        schedule=''

    if ValExist(vacancy,'employment'):
        employment=vacancy['employment']['name']
    else:
        employment=''

    if ValExist(vacancy,'description'):
        reg=r'</li>|<li>|<strong>|</strong>|<br />|</ul>|<ul>|</p>|<p>|<hr />|<ol>|<em>|</em>'
        description=re.sub(reg,'',vacancy['description'])
    else:
        description=''

    if ValExist(vacancy,'key_skills'):
        key_skills=''
        for ks in vacancy['key_skills']:
            key_skills+=ks['name']+' '
    else:
        key_skills=''

    if ValExist(vacancy,'specializations'):
        specializations=set({})
        for v in vacancy['specializations']:
            specializations.add(v['name'])
            specializations.add(v['profarea_name'])
        specializations=' '.join(list(specializations))
    else:
        specializations=''

    if ValExist(vacancy,'professional_roles'):
        professional_roles=set({})
        for v in vacancy['professional_roles']:
            professional_roles.add(v['name'])
        professional_roles=' '.join(list(professional_roles))
    else:
        professional_roles=''

    if ValExist(vacancy,'employer'):
        employer=vacancy['employer']['name']
    else:
        employer=''

    if ValExist(vacancy,'working_time_intervals'):
        working_time_intervals=set({})
        for v in vacancy['working_time_intervals']:
            working_time_intervals.add(v['name'])
        working_time_intervals=' '.join(list(working_time_intervals))
    else:
        working_time_intervals=''

    agg_vac= name + ' ' + area + ' ' + currency + ' ' + address + ' ' + experience + ' ' + schedule + ' ' + employment \
                     + ' ' + description + ' ' + key_skills + ' ' + specializations + ' ' + professional_roles \
                     + ' ' + employer + ' ' + working_time_intervals
    
    
    agg_description.append(cleaner_1(agg_vac))

dfvac=pd.DataFrame({'id_vacancies':vacancies_id[:cnt_vac],'full_description':agg_description})

#### Примеры резюме

In [8]:
resume_1="""Образование: сентябрь 1996 г. – июнь 1998 г. Техникум учёта и механизации, финансовый факультет, специальность – «документоведение», диплом младшего специалиста (дневное отделение). 
Дополнительное образование: январь 2007 г. – апрель 2007 г. Курсы по изучению программы 1С. 

Опыт работы: 

Архивариус август 1998 г. – сентябрь 2005 г. ООО «Оптокомг», г. Киров 
Функциональные обязанности: 
— хранение архивных документов; 
— регистрация и систематизация архивных документов; 
— выдача документов и контроль их возврата; 
— периодическая инвентаризация. 

Кладовщик сентябрь 2005 г. – май 2008 г. ПАО «Спец-торг», г. Киров
Функциональные обязанности: 
— приём, хранение и отпуск товарно-материальных ценностей; 
— обеспечение сохранности ценностей на складе; 
— учёт складских операций; 
— составление отчётности. 

Кладовщик июль 2008 г. – февраль 2014 г. ПАО «Холдинг-групп», г. Киров
Функциональные обязанности: 
— ведение учёта складских операций; 
— проведение разгрузочно-погрузочной работы с продуктами питания;
— обеспечение целостности и сохранности материальных ценностей на складе; 
— обеспечение правильной и безопасной эксплуатации оборудования. 

Профессиональные навыки: 
— Уверенный пользователь ПК, знание офисных программ, в т.ч. 1С; 
— Владение офисной техникой (сканер, принтер, факс); 
— Знание норм складского учёта; 
— Знание складской логистики; 
— Владение языками: русский свободно; английский – со словарём. 

Личные качества: 
— ответственность;
— усидчивость; 
— организованность; 
— честность; 
— внимательность к мелочам; 
— хорошая память. 

Дополнительные сведения: 
Семейное положение: женат. 
Дети: есть. 
Возможность командировок: нет."""


In [77]:
resume_2="""
Менеджер по продажам
Продажи
FMCG, товары народного потребления
Дистрибуция
Прямые продажи
Занятость: полная занятость
График работы: полный день
Желательное время в пути до работы: не имеет значения
Опыт работы — 9 лет 8 месяцев
Апрель 2013 — Август 20174 года 5 месяцев
Дистрибьютор товаров для бизнеса
Москва
Менеджер по продажам
Основные обязанности:
Организация поиска и привлечения клиентов.
Проведение переговоров и заключение договоров.
Проведение маркетинговых активностей.
Контроль дебиторской задолженности.
Выполнение плановых показателей: объем продаж, дебиторская задолженность, активная клиентская база.
Достижения:
Стабильно выполнял поставленные плановые показатели:
ежегодный рост объема продаж на 10–15% за счет развития клиентской базы и расширения представленности продукции компании;
рост активной клиентской базы на 40%;
сокращение просроченной дебиторской задолженности с 30 до 10% за счет усиления контроля финансовой дисциплины клиентов.
Выполнял функции наставника, обучил 3 менеджеров по продажам.
Август 2010 — Октябрь 20133 года 3 месяца
Торгово-производственная компания
Москва
Торговый представитель (10.2010 – 01.2011 мерчандайзер)
Основные обязанности:
Организация продаж на территории (юго-запад Ленинградской области).
Выполнение поставленных целей по планам продаж, ассортименту, установке дополнительного оборудования.
Взаимодействие с клиентами традиционной розницы и локальными ключевыми клиентами.
Организация и проведение маркетинговых мероприятий.
Контроль запасов у локальных ключевых клиентов, помощь в формировании заявок.
Контроль работы мерчандайзеров.
Достижения:
На позиции торгового представителя выполнял поставленные цели:
рост объема продаж 2010/2011 +8%, 2011/2012 +11% за счет развития территории г. Кингисепп и Ивангород;
увеличение представленности продукции компании: + 2 SKU в сети «Ромашка», +3 SKU в сети «Лютик»;
вывод на рынок нового бренда «Мишка» и построение дистрибуции 80% на территории;
перевыполнение целей по установке дополнительного оборудования: в сетевых клиентах — расположение на лучших местах, в традиционной рознице — увеличение дополнительных мест продаж в точках продаж с 30 до 50%.
Неоднократно побеждал в мотивационных программах по выполнению показателей, в 2012 году был признан лучшим сотрудником Северо-Западного региона.
На позиции мерчандайзера выполнял поставленные задачи, улучшил представленность компании в сети «Ромашка» — размещение бесплатных дополнительных мест продаж.
Ключевые навыки
Поиск и привлечение клиентов Ведение переговоров Холодные звонки Активные продажи Управление продажами Планирование продаж Развитие продаж Аналитика продаж Управление отношениями с клиентами Контроль дебиторской задолженности Торговый маркетинг
Рекомендации
по запросу
Обо мне
Опыт работы в сфере продаж более 7 лет.
Ключевые компетенции:
навыки продаж и ведения переговоров;
развитие территории с нуля;
развитие клиентской базы;
расширение представленности продукции компании;
управление продажами;
контроль дебиторской задолженности;
аналитика продаж;
наставничество и обучение новых сотрудников;
ориентация на результат.
Уверенный пользователь ПК: MS Office (Word, Excel, Power Point), 1С, CRM.
Высшее образование
Дата окончания вуза
Полное наименование вуза
Факультет \ специальность
Знание языков
Русский — родной
Английский — уровень владения языком
Повышение квалификации, курсы
2014
Ведение переговоров
Внутренний тренинг дистрибьютора товаров народного потребления
2011
Управление территорией
Внутренний тренинг торгово-производственной компании
2010
Навыки продаж
Внутренний тренинг торгово-производственной компании

"""

In [63]:
resume_3="""Программист высокого уровня. C#,C++,JavaScript,Python. Бэкенд разработка."""

In [106]:
resume_4="""Грумер
40 000 руб.
Специализации:
Другое
Занятость: частичная занятость, полная занятость
График работы: гибкий график, полный день, сменный график
Опыт работы 6 лет 4 месяца
Март 2016 — по настоящее время
6 лет 4 месяца
.
Москва
Грумер
Ключевые навыки
MS Excel
MS Word
Зоопсихолог
Кинолог
Передержка животных
Зоотакси
Опыт вождения
Имеется собственный автомобиль
Права категории B
Обо мне
Здравствуйте! Я дипломированный грумер с большим опытом работы и общения с животными (развивалась и работала в сфере кинологии и зоопсихологии) и людьми. Ищу работу в салоне, предпочтительно ЮВАО.
Аккуратна, целенаправленна, работаю качественно и быстро.
Заранее спасибо.
Образование
Среднее образование
Знание языков
Русский — Родной
Повышение квалификации, курсы
2017
Курс салонного груминга Романа Фомина
Груминг центр Романа Фомина , Грумер-стажёр
Гражданство, время в пути до работы
Гражданство: Россия
Разрешение на работу: Россия
Желательное время в пути до работы: не имеет значения
"""

#### Выбираем одно из резюме и делаем его токенизацию, лемматизацию и чистку от знаков препинания

In [121]:
resume=cleaner_1(resume_4)

#### Объединяем вакансии и резюме для единой векторизации

In [122]:
train_corpus=dfvac['full_description'].values
train_corpus=np.append(train_corpus,resume)

#### Добавляем стоп-слова для исключения из мешка 

In [123]:
stop=stopwords.words('russian')

#### Проводим "обучение" для последующей векторизации вакансий и резюме

In [124]:
#TFIDF
text_transformer_TFIDF=TfidfVectorizer(stop_words=stop)
text_transformer_TFIDF.fit(train_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [125]:
#CountVectorizer
text_transformer_Count=CountVectorizer(stop_words=stop)
text_transformer_Count.fit(train_corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

#### Проводим векторизацию

In [126]:
# "векторизуем" вакансии
#TFIDF
vectorized_vacancies_TFIDF=text_transformer_TFIDF.transform(dfvac['full_description'])
#CountVectiorizer
vectorized_vacancies_Count=text_transformer_Count.transform(dfvac['full_description'])

In [127]:
# "векторизуем" резюме
resume=[resume]
#TFIDF
vectorized_resume_TFIDF=text_transformer_TFIDF.transform(resume)
#CountVectorizer
vectorized_resume_Count=text_transformer_Count.transform(resume)

#### Собираем результат в единый DataFrame

In [128]:
# формируем датафрейм с кодом вакансии и косинусным расстоянием
#TFIDF
TOP_sutable_vac_TFIDF=pd.DataFrame({'id_vacancies':dfvac['id_vacancies'].values,\
                              'cosine_metric':cosine_similarity(vectorized_vacancies_TFIDF,vectorized_resume_TFIDF).T[0]})
#CountVectorizer
TOP_sutable_vac_Count=pd.DataFrame({'id_vacancies':dfvac['id_vacancies'].values,\
                              'cosine_metric':cosine_similarity(vectorized_vacancies_Count,vectorized_resume_Count).T[0]})

In [129]:
# сортируем вакансии по убыванию "близости" к резюме
#TFIDF
TOP_sutable_vac_TFIDF.sort_values(by='cosine_metric',ascending=False,inplace=True)
#Count
TOP_sutable_vac_Count.sort_values(by='cosine_metric',ascending=False,inplace=True)

In [130]:
TOP_sutable_vac_TFIDF.head()

,id_vacancies,cosine_metric
219,55266705,0.133604
429,55578798,0.098854
114,55489486,0.096849
1728,66108794,0.091837
1061,55585881,0.091315


In [131]:
TOP_sutable_vac_Count.head()

,id_vacancies,cosine_metric
213,55268471,0.449675
1939,55163414,0.442075
1441,55265680,0.421246
1791,55347518,0.416141
1905,55476697,0.415792


In [132]:
# выбираем ТОП=10 вакансий и выводим их названия и описания
#TFIDF
TOP=10
for i in range(TOP):
    
    url_one=f'https://api.hh.ru/vacancies/{TOP_sutable_vac_TFIDF.iloc[i,0]}'

    res_one=requests.get(url_one)

    vacancy=res_one.json()
    print('{0}:\n{1} \n\n'.format(vacancy['name'],re.sub(reg,'',vacancy['description'])))

Специалист по уходу за животными в приюте:
Обязанности: - Ежедневный обход и осмотр животных; - Выполнение назначений ветеринарного врача; - Выполнение массажа и реабилитации животных; - Следить за порядком и чистотой - Общаться с гостями приюта. Требования: - любовь к животным; - чистоплотность; - Стрессоустойчивость; - Ответственность; - Внимательность. Условия:  5/2, с 09.00 до 18.00 Есть возможность проживания 40000 р, возможность роста и повышения  


Администратор:
Обязанности:  Осуществление эффективного и культурного обслуживания посетителей, ответы на обращения Знание и ведение, кассовой дисциплины в соответствии с законодательством РБ, Контроль за сохранностью ТМЦ  Требования:  Клиентоориентированность Работа в режиме многозадачности Умение выходить из конфликтных ситуаций, стрессоустойчивость Умение работать в команде, коммуникабельность Качественно выполнять поставленные задачи Владение ПК (Excel, Word). работа с crm Опыт работы от 1-ого года  Условия:  График работы 5/2 По

In [133]:
# выбираем ТОП=10 вакансий и выводим их названия и описания
#CountVectorizer
TOP=10
for i in range(TOP):
    
    url_one=f'https://api.hh.ru/vacancies/{TOP_sutable_vac_Count.iloc[i,0]}'

    res_one=requests.get(url_one)

    vacancy=res_one.json()
    print('{0}:\n{1} \n\n'.format(vacancy['name'],re.sub(reg,'',vacancy['description'])))

Начинающий IT-менеджер:
Обязанности:  Работать над проектами и задачами компании; Работать с большим объемом информации; Вести отчетность о проделанной работе; Готовы рассмотреть кандидатов без опыта работы.  Требования:  Желание развиваться и работать в IT-сфере; Возможность работать 40 часов в неделю; Коммуникативность, ответственность, исполнительность.  Условия:  Работа удаленно; Частичная или полная занятость (от 30 до 40 часов); Гибкий график; Своевременная выплата зарплат.  


Начинающий IT-специалист/Сборщик:
Обязанности:  Сборка ПК и серверов; Установка операционных систем; Тестирование работоспособности компьютера; Сдача собранного оборудования;  Требования:  Скорость работы; Умение работать в команде; Внимательность к мелочам; Опыт работы не важен, но будет плюсом;  Условия:  Хорошая заработная плата 40 000, + премии и бонусы от компании; Гибкий график работы, 2/2, 3/2, 5/2; Работа в уютном офисе в центре города; Работа в молодом, мужском коллективе.  


Менеджер WhatsApp:
О

In [135]:
df_method_resume_metrics=pd.DataFrame({'Method':['TFIDF','TFIDF','TFIDF','TFIDF','CountVec','CountVec','CountVec','CountVec'],\
                                      'Resume Type':['Программист','Менеджер','Кладовщик','Грумер','Программист','Менеджер','Кладовщик','Грумер'],\
                                      'CntValidVac':[5,10,8,1,6,10,6,0]})

In [136]:
df_method_resume_metrics

,Method,Resume Type,CntValidVac
0,TFIDF,Программист,5
1,TFIDF,Менеджер,10
2,TFIDF,Кладовщик,8
3,TFIDF,Грумер,1
4,CountVec,Программист,6
5,CountVec,Менеджер,10
6,CountVec,Кладовщик,6
7,CountVec,Грумер,0


In [137]:
df_method_resume_metrics['Accuracy']=df_method_resume_metrics['CntValidVac']/10

In [138]:
df_method_resume_metrics

,Method,Resume Type,CntValidVac,Accuracy
0,TFIDF,Программист,5,0.5
1,TFIDF,Менеджер,10,1.0
2,TFIDF,Кладовщик,8,0.8
3,TFIDF,Грумер,1,0.1
4,CountVec,Программист,6,0.6
5,CountVec,Менеджер,10,1.0
6,CountVec,Кладовщик,6,0.6
7,CountVec,Грумер,0,0.0
